In [1]:
import os
import pandas as pd
from tqdm import tqdm
from socket import socket

import pandas as pd
import unidecode
import string
import sklearn.feature_extraction
import itertools
import re
import nltk
import normalise

from langdetect import detect
from tqdm import tqdm
from normalise import normalise

from collections import Counter

import pickle

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.semi_supervised.label_propagation module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.semi_supervised. Anything that cannot be imported from sklearn.semi_supervised is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LabelPropagation from version 0.18 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [ ]:
dic = "/Users/emilythiel/Library/Mobile Documents/com~apple~CloudDocs/Documents/University/2020/ISM_Honours/Research/030110/"
#dic = "/Users/tomashegewisch/Downloads/test/"
files = os.listdir(dic)
df = pd.DataFrame()
for i in tqdm(files):
    #print(dic+str(i))
    try:
        temp_df = pd.read_json(dic+str(i), lines=True, orient='records')
    except:
        print("somthing went wrong with this one..." + str(i))
    df = df.append(temp_df)
print("done")

In [ ]:
dic.to_pickle(file_name)

In [15]:
for dependency in ("brown", "names", "wordnet", "averaged_perceptron_tagger", "universal_tagset"):
    nltk.download(dependency)

[nltk_data] Downloading package brown to
[nltk_data]     /Users/emilythiel/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package names to
[nltk_data]     /Users/emilythiel/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/emilythiel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/emilythiel/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/emilythiel/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [16]:
text = ["On", "the", "28", "Apr.", "2010", ",", "Dr.", "Banks", "bought", "a", "chair", "for", "£35", "."]

normalise(text, verbose=False)

['On',
 'the',
 'twenty-eighth of',
 'April',
 'twenty ten',
 ',',
 'Doctor',
 'Banks',
 'bought',
 'a',
 'chair',
 'for',
 'thirty five pounds',
 '.']

In [20]:
rm_punc = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
stop_words = sklearn.feature_extraction.text.ENGLISH_STOP_WORDS

In [26]:
def clean_tweet_word(word):
    if (len(word) > 1) and (word[0] == '#' or word[0] == '@'):
        return word[0] + word[1:].translate(rm_punc).strip()
    if word.startswith('http') or word.startswith('pic.twitter.com') or word.endswith('.com') or word.endswith('.co.za'):
        return ''
    if word in stop_words:
        return ''
    return word.translate(rm_punc).strip()

def clean_tweet(tweet):
    tweet = unidecode.unidecode(tweet).lower().split()
    tweet = [clean_tweet_word(x) for x in tweet]
    tweet = list(itertools.chain.from_iterable([x.split() for x in tweet if x != '']))
    tweet = [x for x in tweet if len(x) > 1]
    return tweet

In [ ]:
# we want to get rid of all the tweets that do not have english in them

from langdetect import DetectorFactory #to improve consistency on language detection
DetectorFactory.seed = 0

print("The length of the DF(before): " + str(len(df)))
non_text = 0
non_en = 0
temp_list_non_eng = []
for i in tqdm(df['tweet'].tolist()):
    try:
        if detect(i) != "en":
            temp_list_non_eng.append(i)
    except:
        non_text +=1
print("Non-Recognisable text: " + str(non_text)) #emojis etc.         
print("We found: "+str(len(temp_list_non_eng))+ " non-eng tweets")

for i in temp_list_non_eng:
    tweets = tweets[tweets.tweet != i]
    
print("The length of the DF(after): " + str(len(tweets)))

In [24]:
#needs to run just before it is tokenized
stopwords = nltk.corpus.stopwords.words('english')
# RegEx for stopwords
re_stopwords = r'\b(?:{})\b'.format('|'.join(stopwords))
# replace '|'-->' ' and drop all stopwords
words = (df.tweet
           .str.lower()
           .replace([r'\|', re_stopwords], [' ', ''], regex=True)
           .str.cat(sep=' ')
           .translate(rm_punc).strip()
           .split()
)
# generate DF out of Counter
rslt = pd.DataFrame(Counter(words).most_common(100),
                    columns=['Word', 'Frequency']).set_index('Word')
print(rslt)

           Frequency
Word                
com           600066
twitter       563409
https         382506
…             382040
status        336740
...              ...
home            8937
something       8792
beautiful       8569
next            8464
za              8448

[100 rows x 1 columns]
